In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

pwd = os.getcwd()

In [2]:
nodes = np.array([8,16,32,64,128,256])

In [3]:
times_map = pd.DataFrame(
    {
        "files" : [],
        "server-nodes-fraction" : [],
        "server-rpc-xstreams-per-MPI-rank" : [],
        "total-num-nodes" : [],
        "MPI-rank-among-clients" : [],
        "run-number" : [],
        "pep_start_times" : [],
        "pep_end_times" : [],
        "analysis_end_times" : [],
        "analysis-method" : [],
    }
)

In [4]:
#1929-files

#8,16,32,64,128,256
es1929_pep_start_times = [[],[],[],[],[],[]]
es1929_pep_end_times = [[],[],[],[],[],[]]
es1929_analysis_end_times = [[],[],[],[],[],[]]
es1929_mpi_rank_among_clients = [[],[],[],[],[],[]]

# 8 node run results:
results_8_nodes = str("/work1/fwk/sajid/hepnos_es/es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/es_1929/es_1929_10_13_22")
with os.scandir(results_8_nodes) as it:
    _pep_start_times = []
    _pep_end_times = []
    _analysis_end_times = []
    _mpi_rank_among_clients = []
    for entry in it:
        if entry.name.startswith('timing') and entry.is_file():
            f = open(entry)
            l = f.readlines()
            for k in range(len(l)):
                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                    _start_time = float(l[k].split(",")[0])
                    _end_time = float(l[k+1].split(",")[0])
                    _pep_start_times.append(_start_time)
                    _pep_end_times.append(_end_time)
                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
    _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
    # Skip failed runs
    if(_time_avg > 0):
        print("run with {} nodes, avg time is {}".format(8, _time_avg))
        es1929_pep_start_times[0].append(_pep_start_times)
        es1929_pep_end_times[0].append(_pep_end_times)
        es1929_analysis_end_times[0].append(_analysis_end_times)
        es1929_mpi_rank_among_clients[0].append(_mpi_rank_among_clients)
        
_rows = len(np.array(es1929_pep_start_times[0]).flatten())
_df_8_ranks = pd.DataFrame(
    {
        "pep_start_times": np.array(es1929_pep_start_times[0]).flatten(),
        "pep_end_times" : np.array(es1929_pep_end_times[0]).flatten(),
        "analysis_end_times": np.array(es1929_analysis_end_times[0]).flatten(),
        "MPI-rank-among-clients" : np.array(es1929_mpi_rank_among_clients[0]).flatten(),
        "run-number" : np.ones(_rows, dtype=np.int32)*1,
        "files" : np.ones(_rows, dtype=np.int32)*1929,
        "total-num-nodes" : np.ones(_rows, dtype=np.int32)*8,
        "server-nodes-fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
        "server-rpc-xstreams-per-MPI-rank" : np.ones(_rows, dtype=np.int32)*16,
        "analysis-method" : str("HEPnOS-rocksdb"),
    }
)
times_map = pd.concat([times_map, _df_8_ranks], ignore_index=True)

map_results_path = str("/work1/fwk/sajid/hepnos_es/scaling/hepnos/servers_one_eigth/es_1929/16MPIranks/")

#16,32,64,128,256
with os.scandir(map_results_path) as it:
    dirs = []
    for _dir in it:
        if not _dir.name.startswith('es') and _dir.is_dir():
            dirs.append(_dir)
    if len(dirs)==0:
        dirs.append(map_results_path)
    for _dir in dirs:
        for i in range(5):
            os.chdir(_dir)
            _nodes = nodes[i+1]
            # typo in submit script!
            _path = os.path.join(os.getcwd()+"/es_7716_"+str(_nodes))
            for j in np.arange(1,6):
                os.chdir(_path+"/RUN_"+str(j))
                _pep_start_times = []
                _pep_end_times = []
                _analysis_end_times = []
                _mpi_rank_among_clients = []
                with os.scandir(os.getcwd()) as it:
                    for entry in it:
                        if entry.name.startswith('timing') and entry.is_file():
                            f = open(entry)
                            l = f.readlines()
                            for k in range(len(l)):
                                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                                    _start_time = float(l[k].split(",")[0])
                                    _end_time = float(l[k+1].split(",")[0])
                                    _pep_start_times.append(_start_time)
                                    _pep_end_times.append(_end_time)
                                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
                _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
                # Skip failed runs
                if(_time_avg > 0):
                    print("run with {} nodes, avg time is {}".format(_nodes, _time_avg))
                    es1929_pep_start_times[i+1].append(_pep_start_times)
                    es1929_pep_end_times[i+1].append(_pep_end_times)
                    es1929_analysis_end_times[i+1].append(_analysis_end_times)
                    es1929_mpi_rank_among_clients[i+1].append(_mpi_rank_among_clients)
            _rows = len(np.array(es1929_pep_start_times[i+1]).flatten())
            _df = pd.DataFrame(
                {
                    "pep_start_times": np.array(es1929_pep_start_times[i+1]).flatten(),
                    "pep_end_times" : np.array(es1929_pep_end_times[i+1]).flatten(),
                    "analysis_end_times": np.array(es1929_analysis_end_times[i+1]).flatten(),
                    "MPI-rank-among-clients" : np.array(es1929_mpi_rank_among_clients[i+1]).flatten(),
                    "run-number" : np.ones(_rows, dtype=np.int32)*j,
                    "files" : np.ones(_rows, dtype=np.int32)*1929,
                    "total-num-nodes" : np.ones(_rows, dtype=np.int32)*_nodes,
                    "server-nodes-fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
                    "server-rpc-xstreams-per-MPI-rank" : np.ones(_rows, dtype=np.int32)*16,
                    "analysis-method" : str("HEPnOS-rocksdb"),
                }
            )
            times_map = pd.concat([times_map, _df], ignore_index=True)

run with 8 nodes, avg time is 195.4054844642857
run with 16 nodes, avg time is 96.56530142857142
run with 16 nodes, avg time is 96.74776924107142
run with 16 nodes, avg time is 96.52992004464285
run with 16 nodes, avg time is 96.73902258928571
run with 16 nodes, avg time is 96.41631044642858
run with 32 nodes, avg time is 52.798309709821424
run with 32 nodes, avg time is 53.748558124999995
run with 32 nodes, avg time is 52.01130984375
run with 32 nodes, avg time is 53.03640950892857
run with 32 nodes, avg time is 52.83598640625001
run with 64 nodes, avg time is 38.91749228794644
run with 64 nodes, avg time is 37.47816463169643
run with 64 nodes, avg time is 35.242412578125005
run with 64 nodes, avg time is 38.04031599330357
run with 64 nodes, avg time is 39.97089716517856
run with 128 nodes, avg time is 32.54480866071429
run with 128 nodes, avg time is 33.577550658482146
run with 128 nodes, avg time is 32.642126964285715
run with 128 nodes, avg time is 31.601366562499997
run with 128 n

In [5]:
#3858-files

#8,16,32,64,128,256
es3858_pep_start_times = [[],[],[],[],[],[]]
es3858_pep_end_times = [[],[],[],[],[],[]]
es3858_analysis_end_times = [[],[],[],[],[],[]]
es3858_mpi_rank_among_clients = [[],[],[],[],[],[]]

# 8 node run results
results_8_nodes = str("/work1/fwk/sajid/hepnos_es/es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/es_3858/es_3858_10_13_22")
with os.scandir(results_8_nodes) as it:
    _pep_start_times = []
    _pep_end_times = []
    _analysis_end_times = []
    _mpi_rank_among_clients = []
    for entry in it:
        if entry.name.startswith('timing') and entry.is_file():
            f = open(entry)
            l = f.readlines()
            for k in range(len(l)):
                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                    _start_time = float(l[k].split(",")[0])
                    _end_time = float(l[k+1].split(",")[0])
                    _pep_start_times.append(_start_time)
                    _pep_end_times.append(_end_time)
                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
    _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
    # Skip failed runs
    if(_time_avg > 0):
        print("run with {} nodes, avg time is {}".format(8, _time_avg))
        es3858_pep_start_times[0].append(_pep_start_times)
        es3858_pep_end_times[0].append(_pep_end_times)
        es3858_analysis_end_times[0].append(_analysis_end_times)
        es3858_mpi_rank_among_clients[0].append(_mpi_rank_among_clients)
        
_rows = len(np.array(es3858_pep_start_times[0]).flatten())
_df_8_ranks = pd.DataFrame(
    {
        "pep_start_times": np.array(es3858_pep_start_times[0]).flatten(),
        "pep_end_times" : np.array(es3858_pep_end_times[0]).flatten(),
        "analysis_end_times": np.array(es3858_analysis_end_times[0]).flatten(),
        "MPI-rank-among-clients" : np.array(es3858_mpi_rank_among_clients[0]).flatten(),
        "run-number" : np.ones(_rows, dtype=np.int32)*1,
        "files" : np.ones(_rows, dtype=np.int32)*3858,
        "total-num-nodes" : np.ones(_rows, dtype=np.int32)*8,
        "server-nodes-fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
        "server-rpc-xstreams-per-MPI-rank" : np.ones(_rows, dtype=np.int32)*16,
        "analysis-method" : str("HEPnOS-rocksdb"),
    }
)
times_map = pd.concat([times_map, _df_8_ranks], ignore_index=True)

map_results_path = str("/work1/fwk/sajid/hepnos_es/scaling/hepnos/servers_one_eigth/es_3858/16MPIranks/")

#16,32,64,128,256
with os.scandir(map_results_path) as it:
    dirs = []
    for _dir in it:
        if not _dir.name.startswith('es') and _dir.is_dir():
            dirs.append(_dir)
    if len(dirs)==0:
        dirs.append(map_results_path)
    for _dir in dirs:
        for i in range(5):
            os.chdir(_dir)
            _nodes = nodes[i+1]
            # typo in submit script!
            _path = os.path.join(os.getcwd()+"/es_7716_"+str(_nodes))
            for j in np.arange(1,6):
                os.chdir(_path+"/RUN_"+str(j))
                _pep_start_times = []
                _pep_end_times = []
                _analysis_end_times = []
                _mpi_rank_among_clients = []
                with os.scandir(os.getcwd()) as it:
                    for entry in it:
                        if entry.name.startswith('timing') and entry.is_file():
                            f = open(entry)
                            l = f.readlines()
                            for k in range(len(l)):
                                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                                    _start_time = float(l[k].split(",")[0])
                                    _end_time = float(l[k+1].split(",")[0])
                                    _pep_start_times.append(_start_time)
                                    _pep_end_times.append(_end_time)
                                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
                _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
                # Skip failed runs
                if(_time_avg > 0):
                    print("run with {} nodes, avg time is {}".format(_nodes, _time_avg))
                    es3858_pep_start_times[i+1].append(_pep_start_times)
                    es3858_pep_end_times[i+1].append(_pep_end_times)
                    es3858_analysis_end_times[i+1].append(_analysis_end_times)
                    es3858_mpi_rank_among_clients[i+1].append(_mpi_rank_among_clients)
            _rows = len(np.array(es3858_pep_start_times[i+1]).flatten())
            _df = pd.DataFrame(
                {
                    "pep_start_times": np.array(es3858_pep_start_times[i+1]).flatten(),
                    "pep_end_times" : np.array(es3858_pep_end_times[i+1]).flatten(),
                    "analysis_end_times": np.array(es3858_analysis_end_times[i+1]).flatten(),
                    "MPI-rank-among-clients" : np.array(es3858_mpi_rank_among_clients[i+1]).flatten(),
                    "run-number" : np.ones(_rows, dtype=np.int32)*j,
                    "files" : np.ones(_rows, dtype=np.int32)*3858,
                    "total-num-nodes" : np.ones(_rows, dtype=np.int32)*_nodes,
                    "server-nodes-fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
                    "server-rpc-xstreams-per-MPI-rank" : np.ones(_rows, dtype=np.int32)*16,
                    "analysis-method" : str("HEPnOS-rocksdb"),
                }
            )
            times_map = pd.concat([times_map, _df], ignore_index=True)

run with 8 nodes, avg time is 392.1498567857143
run with 16 nodes, avg time is 192.6773599107143
run with 16 nodes, avg time is 191.91256767857143
run with 16 nodes, avg time is 191.6889172767857
run with 16 nodes, avg time is 191.3910180357143
run with 16 nodes, avg time is 191.80310433035714
run with 32 nodes, avg time is 102.99635223214285
run with 32 nodes, avg time is 101.25129287946427
run with 32 nodes, avg time is 101.40449584821428
run with 32 nodes, avg time is 100.71287258928571
run with 32 nodes, avg time is 99.13890283482144
run with 64 nodes, avg time is 68.30175345982143
run with 64 nodes, avg time is 69.14729272321429
run with 64 nodes, avg time is 69.71433803571429
run with 64 nodes, avg time is 70.31099247767858
run with 64 nodes, avg time is 70.092546640625
run with 128 nodes, avg time is 55.81195737723215
run with 128 nodes, avg time is 52.97703584821428
run with 128 nodes, avg time is 53.203388627232144
run with 128 nodes, avg time is 56.69837511160714
run with 128

In [6]:
#7716-files

#8,16,32,64,128,256
es7716_pep_start_times = [[],[],[],[],[],[]]
es7716_pep_end_times = [[],[],[],[],[],[]]
es7716_analysis_end_times = [[],[],[],[],[],[]]
es7716_mpi_rank_among_clients = [[],[],[],[],[],[]]

# 8 node run results are at:
results_8_nodes = str("/work1/fwk/sajid/hepnos_es/es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/es_7716/es_7716_10_13_22")
with os.scandir(results_8_nodes) as it:
    _pep_start_times = []
    _pep_end_times = []
    _analysis_end_times = []
    _mpi_rank_among_clients = []
    for entry in it:
        if entry.name.startswith('timing') and entry.is_file():
            f = open(entry)
            l = f.readlines()
            for k in range(len(l)):
                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                    _start_time = float(l[k].split(",")[0])
                    _end_time = float(l[k+1].split(",")[0])
                    _pep_start_times.append(_start_time)
                    _pep_end_times.append(_end_time)
                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
    _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
    # Skip failed runs
    if(_time_avg > 0):
        print("run with {} nodes, avg time is {}".format(8, _time_avg))
        es7716_pep_start_times[0].append(_pep_start_times)
        es7716_pep_end_times[0].append(_pep_end_times)
        es7716_analysis_end_times[0].append(_analysis_end_times)
        es7716_mpi_rank_among_clients[0].append(_mpi_rank_among_clients)
        
_rows = len(np.array(es7716_pep_start_times[0]).flatten())
_df_8_ranks = pd.DataFrame(
    {
        "pep_start_times": np.array(es7716_pep_start_times[0]).flatten(),
        "pep_end_times" : np.array(es7716_pep_end_times[0]).flatten(),
        "analysis_end_times": np.array(es7716_analysis_end_times[0]).flatten(),
        "MPI-rank-among-clients" : np.array(es7716_mpi_rank_among_clients[0]).flatten(),
        "run-number" : np.ones(_rows, dtype=np.int32)*1,
        "files" : np.ones(_rows, dtype=np.int32)*7716,
        "total-num-nodes" : np.ones(_rows, dtype=np.int32)*8,
        "server-nodes-fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
        "server-rpc-xstreams-per-MPI-rank" : np.ones(_rows, dtype=np.int32)*16,
        "analysis-method" : str("HEPnOS-rocksdb"),
    }
)
times_map = pd.concat([times_map, _df_8_ranks], ignore_index=True)

map_results_path = str("/work1/fwk/sajid/hepnos_es/scaling/hepnos/servers_one_eigth/es_7716/16MPIranks/")

#16,32,64,128,256
with os.scandir(map_results_path) as it:
    dirs = []
    for _dir in it:
        if not _dir.name.startswith('es') and _dir.is_dir():
            dirs.append(_dir)
    if len(dirs)==0:
        dirs.append(map_results_path)
    for _dir in dirs:
        for i in range(5):
            os.chdir(_dir)
            _nodes = nodes[i+1]
            _path = os.path.join(os.getcwd()+"/es_7716_"+str(_nodes))
            for j in np.arange(1,6):
                os.chdir(_path+"/RUN_"+str(j))
                _pep_start_times = []
                _pep_end_times = []
                _analysis_end_times = []
                _mpi_rank_among_clients = []
                with os.scandir(os.getcwd()) as it:
                    for entry in it:
                        if entry.name.startswith('timing') and entry.is_file():
                            f = open(entry)
                            l = f.readlines()
                            for k in range(len(l)):
                                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                                    _start_time = float(l[k].split(",")[0])
                                    _end_time = float(l[k+1].split(",")[0])
                                    _pep_start_times.append(_start_time)
                                    _pep_end_times.append(_end_time)
                                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
                _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
                # Skip failed runs
                if(_time_avg > 0):
                    print("run with {} nodes, avg time is {}".format(_nodes, _time_avg))
                    es7716_pep_start_times[i+1].append(_pep_start_times)
                    es7716_pep_end_times[i+1].append(_pep_end_times)
                    es7716_analysis_end_times[i+1].append(_analysis_end_times)
                    es7716_mpi_rank_among_clients[i+1].append(_mpi_rank_among_clients)
            _rows = len(np.array(es7716_pep_start_times[i+1]).flatten())
            _df = pd.DataFrame(
                {
                    "pep_start_times": np.array(es7716_pep_start_times[i+1]).flatten(),
                    "pep_end_times" : np.array(es7716_pep_end_times[i+1]).flatten(),
                    "analysis_end_times": np.array(es7716_analysis_end_times[i+1]).flatten(),
                    "MPI-rank-among-clients" : np.array(es7716_mpi_rank_among_clients[i+1]).flatten(),
                    "run-number" : np.ones(_rows, dtype=np.int32)*j,
                    "files" : np.ones(_rows, dtype=np.int32)*7716,
                    "total-num-nodes" : np.ones(_rows, dtype=np.int32)*_nodes,
                    "server-nodes-fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
                    "server-rpc-xstreams-per-MPI-rank" : np.ones(_rows, dtype=np.int32)*16,
                    "analysis-method" : str("HEPnOS-rocksdb"),
                }
            )
            times_map = pd.concat([times_map, _df], ignore_index=True)

run with 8 nodes, avg time is 780.9602066071428
run with 16 nodes, avg time is 384.5551071428572
run with 16 nodes, avg time is 384.34842084821423
run with 16 nodes, avg time is 384.3167251785714
run with 16 nodes, avg time is 384.69043732142865
run with 16 nodes, avg time is 384.48889236607147
run with 32 nodes, avg time is 194.78196714285716
run with 32 nodes, avg time is 196.5562596875
run with 32 nodes, avg time is 195.62979424107144
run with 32 nodes, avg time is 193.96487839285712
run with 32 nodes, avg time is 196.10852401785718
run with 64 nodes, avg time is 130.34335783482143
run with 64 nodes, avg time is 131.02717642857144
run with 64 nodes, avg time is 127.86332690848214
run with 64 nodes, avg time is 129.33177765625
run with 64 nodes, avg time is 134.79486690848213
run with 128 nodes, avg time is 98.57186023437498
run with 128 nodes, avg time is 96.95414957589287
run with 128 nodes, avg time is 99.28946146763393
run with 128 nodes, avg time is 94.70769714285714
run with 12

In [7]:
times_map

,files,server-nodes-fraction,server-rpc-xstreams-per-MPI-rank,total-num-nodes,MPI-rank-among-clients,run-number,pep_start_times,pep_end_times,analysis_end_times,analysis-method
0,1929.0,0.125,16.0,8.0,107.0,1.0,1.50938,196.9850,199.591,HEPnOS-rocksdb
1,1929.0,0.125,16.0,8.0,111.0,1.0,1.50936,196.8690,199.591,HEPnOS-rocksdb
2,1929.0,0.125,16.0,8.0,45.0,1.0,1.49809,196.9340,199.549,HEPnOS-rocksdb
3,1929.0,0.125,16.0,8.0,36.0,1.0,1.49808,196.9820,199.567,HEPnOS-rocksdb
4,1929.0,0.125,16.0,8.0,53.0,1.0,1.49566,196.7790,199.555,HEPnOS-rocksdb
...,...,...,...,...,...,...,...,...,...,...
104491,7716.0,0.125,16.0,256.0,2001.0,5.0,3.28214,92.8181,102.764,HEPnOS-rocksdb
104492,7716.0,0.125,16.0,256.0,1509.0,5.0,3.32572,93.1997,102.828,HEPnOS-rocksdb
104493,7716.0,0.125,16.0,256.0,1613.0,5.0,3.30330,93.5123,102.789,HEPnOS-rocksdb
104494,7716.0,0.125,16.0,256.0,2275.0,5.0,3.29968,95.0132,102.765,HEPnOS-rocksdb


In [8]:
os.chdir(pwd)
times_map.to_csv("map-hepnos.csv")